In [151]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import matplotlib.pyplot as plt

In [304]:
df = pd.read_csv("../Cases/Glass Identification/Glass.csv")
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,building_windows_float_processed
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,building_windows_float_processed
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,building_windows_float_processed
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,building_windows_float_processed
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,building_windows_float_processed
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,headlamps
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,headlamps
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,headlamps
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,headlamps


In [305]:
X = df.drop('Type', axis=1)
y = df['Type']

In [306]:
le = LabelEncoder()
le_y = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X,le_y, test_size = 0.3, 
                                                    random_state=24,stratify=le_y)

In [307]:
X.shape, y.shape

((214, 9), (214,))

In [308]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
y_trn_ohe = ohe.fit_transform(y_train.reshape(-1,1))
y_trn_ohe.shape

(149, 6)

In [309]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scl_trn = scaler.fit_transform(X_train)
X_scl_tst = scaler.fit_transform(X_test)

In [310]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_trn_ohe)

In [348]:
torch.manual_seed(24)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=7),
                    nn.SELU(),
                     nn.Linear(in_features=7, out_features=6),
                    nn.SELU(),
                     )

In [349]:
summary(model, (1,X_scl_trn.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 7]              70
              SELU-2                 [-1, 1, 7]               0
            Linear-3                 [-1, 1, 6]              48
              SELU-4                 [-1, 1, 6]               0
Total params: 118
Trainable params: 118
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [350]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr = 0.1)
optimizer

Adagrad (
Parameter Group 0
    differentiable: False
    eps: 1e-10
    foreach: None
    fused: None
    initial_accumulator_value: 0
    lr: 0.1
    lr_decay: 0
    maximize: False
    weight_decay: 0
)

In [351]:
# for p in model.parameters():
#     print(p)

y_pred = model(X_torch.float())
# y_pred[:3]

# y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([149, 6])
torch.Size([149, 6])


In [352]:
y_pred_prob.shape

(65, 6)

In [353]:
for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  1.6923178434371948
epoch:  101  loss:  0.7861059308052063
epoch:  201  loss:  0.7370367646217346
epoch:  301  loss:  0.6978946328163147
epoch:  401  loss:  0.6677765250205994
epoch:  501  loss:  0.6383868455886841
epoch:  601  loss:  0.6224290728569031
epoch:  701  loss:  0.6096880435943604
epoch:  801  loss:  0.5980682373046875
epoch:  901  loss:  0.5871026515960693


In [354]:
X_torch_test = torch.from_numpy(X_scl_tst)

y_wt_sum = model(X_torch_test.float())
softmax = nn.Softmax(dim=1)
y_pred_prob = softmax(y_wt_sum)
y_pred_prob[:3]

tensor([[9.8408e-01, 1.0016e-02, 5.9101e-06, 5.6042e-06, 5.6041e-06, 5.8863e-03],
        [2.0222e-09, 1.4886e-07, 2.6625e-07, 1.0000e+00, 3.4836e-06, 2.0222e-09],
        [2.6641e-01, 5.7006e-01, 5.7092e-04, 5.8399e-04, 1.3203e-02, 1.4917e-01]],
       grad_fn=<SliceBackward0>)

In [355]:
y_pred_prob =y_pred_prob.detach().numpy()
y_pred_prob.shape

(65, 6)

In [356]:
y_pred = np.argmax(y_pred_prob,axis=1)
y_pred.shape

(65,)

In [357]:
accuracy_score(y_test, y_pred)

0.5230769230769231

In [346]:
log_loss(y_test, y_pred_prob)

1.0426343254118517